In [1]:
import ast
import pandas as pd
from dotenv import load_dotenv
import os
import sys
sys.path.append('../')

import src.geo_functions as gf
import src.flights_functions as ff

In [18]:
from geopy.geocoders import Nominatim
from geopy.distance import geodesic 
from pymongo import MongoClient
import json
from bson.json_util import dumps
import pandas as pd
import re
import unicodedata
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
import time
import pandas as pd
from selenium.webdriver.support.ui import Select
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.keys import Keys
from deep_translator import (MyMemoryTranslator)
import pandas as pd
from pymongo import MongoClient
from pymongo import GEOSPHERE
import json
from bson.json_util import dumps
import ast
           

In [6]:
df=gf.df_geonear("Madrid", 800)

/home/lydia/anaconda3/envs/ironhack/lib/python3.8/site-packages/pandas/core/frame.py:4906: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(


kms=list(df.City)

In [9]:
kms

['Córdoba',
 'Valencia',
 'Seville',
 'Andorra la Vella',
 'Lisbon',
 'Barcelona',
 'Algiers',
 'Rabat']

In [16]:
#lista_va=[]
#for k in kms:
    #print(k)
    #dest=ff.get_cheapest_price("Madrid",k,"2021-12-18")
    #lista_va.append(dest)

### IATA CODES:

In [3]:
df=gf.dicc_to_df() 

../src/geo_functions.py:147: UserWarning: Pandas doesn't allow columns to be created via a new attribute name - see https://pandas.pydata.org/pandas-docs/stable/indexing.html#attribute-access
  df.Cities=df.Lugar.apply(lambda x: x.split(",")[0])
../src/geo_functions.py:148: UserWarning: Pandas doesn't allow columns to be created via a new attribute name - see https://pandas.pydata.org/pandas-docs/stable/indexing.html#attribute-access
  df.Countries=df.Lugar.apply(lambda x: x.split(",")[1])


In [21]:
df.sample()

In [6]:
not_spain=df[df["Countries"]!="Espana"]

In [7]:
spain=df[df["Countries"]=="Espana"]

In [8]:
not_spain["Cities"]=not_spain.Cities.apply(gf.translator)

In [9]:
not_spain["Countries"]=not_spain.Countries.apply(gf.translator)

In [12]:
IATA = pd.concat([not_spain,spain], axis=0)

In [21]:
IATA.to_csv("../Data/IATAlimpio.csv",index=False)

In [3]:
IATA=pd.read_csv("../Data/IATAlimpio.csv")

In [4]:
location=IATA.Cities.apply(gf.get_coordenadas_tipo_point)

In [5]:
IATA["Type Point"]=location

In [6]:
IATA["Type Point"]=IATA["Type Point"].apply(gf.string_dicc)

In [7]:
IATA.head()

,IATA,Countries,Cities,Type Point
0,AAA,Polinesia frances,Anaa,"{'type': 'Point', 'coordinates': (-145.521919,..."
1,AAD,Somalia,Adado,"{'type': 'Point', 'coordinates': (47.068920688..."
2,AAE,Argelia,Annaba,"{'type': 'Point', 'coordinates': (7.7549272, 3..."
3,AAK,Kiribati,Aranuka,"{'type': 'Point', 'coordinates': (173.62125479..."
4,AAL,Dinamarca,Aalborg,"{'type': 'Point', 'coordinates': (9.9215263, 5..."


In [2]:
#IATA["Type Point"]=IATA["Type Point"].apply(lambda x: ast.literal_eval(x))

In [8]:
IATA.columns=["IATA","Country","City","Location"]

In [9]:
IATA.head()

,IATA,Country,City,Location
0,AAA,Polinesia frances,Anaa,"{'type': 'Point', 'coordinates': (-145.521919,..."
1,AAD,Somalia,Adado,"{'type': 'Point', 'coordinates': (47.068920688..."
2,AAE,Argelia,Annaba,"{'type': 'Point', 'coordinates': (7.7549272, 3..."
3,AAK,Kiribati,Aranuka,"{'type': 'Point', 'coordinates': (173.62125479..."
4,AAL,Dinamarca,Aalborg,"{'type': 'Point', 'coordinates': (9.9215263, 5..."


In [22]:
IATA["Country"]=IATA.Country.apply(lambda x: x.lstrip())

In [10]:
dicc_IATA=IATA.to_dict('records')

In [12]:
lista_final=list(dicc_IATA)

In [16]:
def insert_Mongo():
    """
    Esta función inserta los datos geográficos en Mongo DB.
    """
    client = MongoClient("localhost:27017")
    db = client.get_database("Tu_viaje_ideal_en_un_click")
    coordenadas = db["coordenadas"]  
    coordenadas.insert_many(lista_final)
 

In [19]:
insert_Mongo()